# LangChain表达式 - Langchain Expression Language

In [1]:
import os

API_KEY = os.getenv('UNION_API_KEY')
BASE_URL = os.getenv('UNION_BASE_URL')

In [3]:
from langchain_openai import ChatOpenAI

openai_chat = ChatOpenAI(api_key=API_KEY, model_name="gpt-3.5-turbo", base_url=BASE_URL)

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 创建一个聊天
model = openai_chat

# 定义一个模板，它接受模板变量的字典，并生成一个 PromptValue
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

# 创建一个输出解析器，将任何输入转换成一个字符串输出
output_parser = StrOutputParser()

# 创建一个链条，将模板、模型和输出解析器串联起来
chain = prompt | model | output_parser

# 将 topic 传递到模板中，并调用链条
chain.invoke({"topic": "ice cream"})



'Why did the ice cream truck break down? Because it had too many "scoops"!'

# RAG示例

In [5]:
# Requires:
# pip install langchain docarray tiktoken

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

# 创建一个DocArrayInMemorySearch对象，并用OpenAIEmbeddings进行初始化
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

# 创建要给RunnableParallel，并行执行retriever和RunnablePassthrough
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")

c:\Users\dell\anaconda3\envs\llm-demo\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


'Harrison worked at Kensho.'